<a href="https://colab.research.google.com/github/graylan0/quantum-machine-learning/blob/main/Quantum_Machine_Learning_gpt_3_5_turbo_pennylane_Weather_Prediction_Hurricane_Warning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pennylane
!pip install requests
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.2
    Uninstalling autograd-1.6.2:
      Successfully uninstalled autograd-1.6.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
import requests
import sqlite3
from datetime import datetime
import openai

openai_api_key = "YOUR_OPENAI_API_KEY_HERE"
openai.api_key = openai_api_key

# Quantum Model
class EnhancedQuantumHurricaneModel:
    def __init__(self, num_qubits, num_layers):
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.dev = qml.device('default.qubit', wires=self.num_qubits)

    def quantum_layer(self, params):
        for i in range(self.num_qubits):
            qml.RY(params[i], wires=i)
        for i in range(self.num_qubits - 1):
            qml.CNOT(wires=[i, i+1])

    def quantum_model(self, params):
        for layer in range(self.num_layers):
            self.quantum_layer(params[layer])

    def encode_weather_data(self, weather_data):
        params = [[weather_data['pressure'], weather_data['temperature'], weather_data['humidity']] for _ in range(self.num_layers)]
        return params

# Fetch Weather Data
def fetch_weather_data(location):
    api_key = "YOUR_API_KEY"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}"
    response = requests.get(base_url)
    weather_data = response.json()
    return {
        'pressure': weather_data['main']['pressure'],
        'temperature': weather_data['main']['temp'],
        'humidity': weather_data['main']['humidity']
    }

# Create SQL Table
def create_table():
    conn = sqlite3.connect('hurricane_predictions.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS predictions
                      (location TEXT, date TEXT, pressure REAL, temperature REAL, humidity REAL, probability REAL)''')
    conn.commit()
    conn.close()

# Store Prediction
def store_prediction(location, weather_data, probability):
    conn = sqlite3.connect('hurricane_predictions.db')
    cursor = conn.cursor()
    date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute("INSERT INTO predictions (location, date, pressure, temperature, humidity, probability) VALUES (?, ?, ?, ?, ?, ?)",
                   (location, date, weather_data['pressure'], weather_data['temperature'], weather_data['humidity'], probability))
    conn.commit()
    conn.close()

# Fetch History
def fetch_history(location):
    conn = sqlite3.connect('hurricane_predictions.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM predictions WHERE location = ?", (location,))
    history = cursor.fetchall()
    conn.close()
    return history

# Analyze History with GPT
def analyze_history_with_gpt(location):
    history = fetch_history(location)
    prompt = f"Analyze the historical weather data and quantum probabilities for {location}:\n"
    for record in history:
        prompt += f"Date: {record[1]}, Pressure: {record[2]} hPa, Temperature: {record[3]} K, Humidity: {record[4]}%, Probability: {record[5]}\n"
    prompt += "Provide insights and predictions for future weather patterns."
    response = openai.ChatCompletion.create(model='gpt-3.5-turbo', prompt=prompt)
    return response.choices[0].text

# Main Function
def predict_hurricane(location):
    weather_data = fetch_weather_data(location)
    qml_model = EnhancedQuantumHurricaneModel(3, 3)  # 3 qubits, 3 layers

    params = qml_model.encode_weather_data(weather_data)

    @qml.qnode(qml_model.dev)
    def circuit(params):
        qml_model.quantum_model(params)
        return qml.expval(qml.PauliZ(wires=0))  # Measure the expectation value of PauliZ on the first qubit

    probability = circuit(params)
    store_prediction(location, weather_data, probability)
    gpt_response = analyze_history_with_gpt(location)

    print("Weather Data:", weather_data)
    print("Quantum Probability:", probability)
    print("GPT Response:", gpt_response)

# Example Usage
create_table()
location = "Florida"
predict_hurricane(location)
